## Introdução

Olá, cientista de dados! Esta é nossa última aula, e para encerrar vamos conhecer uma biblioteca que nos permitirá criar gráficos esteticamente refinados e interativos.

A biblioteca se chama Plotly, e vamos aprender a criar todos os gráficos que criamos com seaborn agora utilizando Plotly!

## Preparando o ambiente 


Aqui, como sempre, vamos carregar as bibliotecas que iremos utilizar no nosso notebook.

In [8]:
import pandas as pd
import plotly.express as px
import duckdb

%reload_ext sql

Vamos utilizar o duckdb em memória caso seja necessário fazer alguma transformação rápida.

In [9]:
%sql duckdb:///:memory:

## Carregando os dados


In [4]:
pokemons = pd.read_csv('https://github.com/labeduc/datasets/blob/main/pokemons/all.csv?raw=true')

pokemons.sample(5)

,Number,Name,Type 1,Type 2,Abilities,HP,Att,Def,Spa,Spd,...,Against Bug,Against Rock,Against Ghost,Against Dragon,Against Dark,Against Steel,Against Fairy,Height,Weight,BMI
100,78,Rapidash,Fire,NaN,"['Flame Body', 'Flash Fire', 'Run Away']",65,100,70,80,80,...,0.5,2.0,1.0,1.0,1.0,0.5,0.5,1.7,95.0,32.9
454,382,Kyogre,Water,NaN,['Drizzle'],100,100,90,150,140,...,1.0,1.0,1.0,1.0,1.0,0.5,1.0,4.5,352.0,17.4
208,165,Ledyba,Bug,Flying,"['Early Bird', 'Rattled', 'Swarm']",40,20,30,40,80,...,0.5,4.0,1.0,1.0,1.0,1.0,1.0,1.0,10.8,10.8
141,109,Koffing,Poison,NaN,"['Levitate', 'Neutralizing Gas', 'Stench']",40,65,95,60,45,...,0.5,1.0,1.0,1.0,1.0,1.0,0.5,0.6,1.0,2.8
18,15,Beedrill,Bug,Poison,"['Sniper', 'Swarm']",65,90,40,45,80,...,0.5,2.0,1.0,1.0,1.0,1.0,0.5,1.0,29.5,29.5


## Tipos de Gráficos

O Plotly tem diversos tipos de gráficos, exatamente como o Seaborn, e iremos ver aqui os tipos mais básicos, assim como vimos com o seaborn nas aulas anteriores. Também já iremos apresentar desde o início os gráficos com as boas práticas devidamente aplicadas, para gerar uma melhor experiência para o público. 

### Barra/Colunas

E lá vamos nós, começando pelo gráfico mais popular de todos, o gráfico de barra/coluna. A princípio, precisamos apenas 2 comandos: o comando que gera o gráfico e armazena em um objeto do tipo `Figure`; e o comando que plota a figura gerada.

In [7]:
# gera a figura chamando a função bar()
# o primeiro parâmetro é o DataFrame e os seguintes 
# são as colunas do DataFrame que serão os eixos X e Y.
fig = px.bar(pokemons, x='Generation', y='Att')

# função para plotar a imagem
fig.show()

Está aí nosso gráfico, mas ele parece estar com um problema: essa aparência riscada. Por que isso aconteceu? Se passarmos o mouse por cima de uma das barras, veremos que na verdade, ela registra cada um dos valores de ataque dos pokemons, ou seja, ele não está calculando a média dos valores, como seria esperado.

E como resolvemos isso? De duas maneiras:
1) fazemos a transformação dos dados, com um group by
2) modificamos o tipo de gráfico.

Nesta aula, vamos resolver com a solução 1 - transformar os dados.

In [19]:
%sql pokemons_gb << select Generation, AVG(Att) as AvgAttack from pokemons group by Generation order by Generation

*  duckdb:///:memory:
Done.


In [20]:
# gera a figura chamando a função bar()
# o primeiro parâmetro é o DataFrame e os seguintes 
# são as colunas do DataFrame que serão os eixos X e Y.
fig = px.bar(pokemons_gb, x='Generation', y='AvgAttack')

# função para plotar a imagem
fig.show()

Está aí nosso gráfico, agora com cada barra representando a média do poder de ataque de cada geração. Para transformar em barras horizontais, basta trocar o eixo X com o Y, além de adicionar o parâmetro de orientação.

In [21]:
# gera a figura chamando a função bar()
# o primeiro parâmetro é o DataFrame e os seguintes 
# são as colunas do DataFrame que serão os eixos X e Y.
fig = px.bar(pokemons_gb, y='Generation', x='AvgAttack', orientation='h')

# função para plotar a imagem
fig.show()

Uma característica do Plotly que é interessante é que ele ocupa toda a largura disponível na tela para plotar o gráfico, e apenas o necessário do ponto de vista de altura. Isto significa que a aparência dos gráficos precisa ser tratada para gerar resultados mais refinados. No seaborn, tinha diversos parâmetros que precisavamos acertar na plotagem do gráfico. Na última seção, veremos o mesmo para o Plotly.

### Linha/Área

E vamos ao gráfico de linha! Esse mesmo conjunto de dados nos permite plotar um gráfico neste estilo.

In [22]:
fig = px.line(pokemons_gb, x="Generation", y="AvgAttack")

fig.show()

Para criar um gráfico de área, basta trocar a função.

In [23]:
fig = px.area(pokemons_gb, x="Generation", y="AvgAttack")

fig.show()

E para os olhos mais treinados, vocês devem ter percebido que o formato do gráfico de linha e do gráfico de área não estão exatamente iguais, o que seria o esperado. O que aconteceu aqui foi uma mudança da escala no eixo do Y, gerando uma discrepância no posicionamento de cada ponto da linha. Por isso conhecer como configurar os gráficos será muito importante.

### Dispersão/Pontos

Este gráfico, que nos permite entender a correlação entre variáveis (até 3, se considerarmos o tamanho), também pode ser criado de forma bem simples. Lembrando que também temos um caso especial de gráfico de dispersão, que é o Quadrante Mágico do Gartner. Este será visto quando falarmos sobre configuração dos gráficos do Plotly.

In [25]:
fig = px.scatter(pokemons, x="Att", y="Def", color="Type 1",
                 size='HP')
fig.show()

### Pizza

E por último, o gráfico que todos amam odiar! Quando estávamos aprendendo seaborn, esse gráfico é o que mais exigia a utilização da MatplotLib e o código era um pouco mais extenso. Aqui, a biblioteca Plotly mantém a simplicidade:

In [35]:
fig = px.pie(pokemons_gb, values='AvgAttack', names='Generation')

fig.show()

E o Plotly também torna mais simples criar uma variação do gráfico de pizza, conhecido como Donut!

In [36]:
fig = px.pie(pokemons_gb, values='AvgAttack', names='Generation', hole=0.45)

fig.show()

## Conclusão

E assim, encerramos a parte básica desta aula. Aprendemos como criar os principais tipos de gráficos usando a biblioteca Plotly.

Aprendemos também que a Plotly permite a criação de gráficos interativos, além de ter uma melhor resolução para apresentar os gráficos. Na última aula do nosso curso, iremos abordar a configuração dos gráficos.

Se você quiser aprender mais sobre Plotly, o link da documentação é este [aqui](https://plotly.com/python/).
